# 13장 인공지능 유튜브 동영상 요약기

## 13.1 유튜브 동영상 자막 가져오기

[13장: 335페이지]

In [ ]:
%%writefile C:\myPyAI\code\my_yt_tran.py
# 유튜브 동영상 정보와 자막을 가져오기 위한 모듈

import openai
import yt_dlp
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
import os
from pathlib import Path

# 유튜브 비디오 정보를 가져오는 함수
def get_youtube_video_info(video_url):
    ydl_opts = {            # 다양한 옵션 지정
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        video_info = ydl.extract_info(video_url, download=False) # 비디오 정보 추출
        video_id = video_info['id']              # 비디오 정보에서 비디오 ID 추출
        title = video_info['title']              # 비디오 정보에서 제목 추출
        upload_date = video_info['upload_date']  # 비디오 정보에서 업로드 날짜 추출
        channel = video_info['channel']          # 비디오 정보에서 채널 이름 추출
        duration = video_info['duration_string']

    return video_id, title, upload_date, channel, duration

# 유튜브 비디오 URL에서 비디오 ID를 추출하는 함수
def get_video_id(video_url):
    video_id = video_url.split('v=')[1][:11]
    
    return video_id 

# 유튜브 동영상 자막을 직접 가져오는 함수
def get_transcript_from_youtube(video_url, lang='en'):
    # 비디오 URL에서 비디오 ID 추출
    video_id = get_video_id(video_url)

    # 자막 리스트 가져오기
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    
#     print(f"- 유튜브 비디오 ID: {video_id}")    
#     for transcript in transcript_list:
#         print(f"- [자막 언어] {transcript.language}, [자막 언어 코드] {transcript.language_code}")

    # 자막 가져오기 (lang)
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])

    text_formatter = TextFormatter() # Text 형식으로 출력 지정
    text_formatted = text_formatter.format_transcript(transcript)
    
    return text_formatted

[13장: 336페이지]

In [ ]:
import my_yt_tran # 유튜브 동영상 정보와 자막을 가져오기 위한 모듈 임포트
import textwrap

video_url = "https://www.youtube.com/watch?v=C_78DM8fG6E&t=6s"

_, video_title, _, _, _ = my_yt_tran.get_youtube_video_info(video_url)
print("- 비디오 제목:", video_title)

yt_transcript = my_yt_tran.get_transcript_from_youtube(video_url, 
                                                       lang='en')
shorten_yt_transcript = textwrap.shorten(yt_transcript, 
                                         250,
                                         placeholder=' [..이하 생략..]')
print("- 자막 내용(축약):", shorten_yt_transcript) # 축약 출력
# print("- 자막 내용:", yt_transcript) # 전체 출력

## 13.2 긴 자막을 분리해 요약하기

[13장: 337페이지]

In [ ]:
import tiktoken

# 텍스트의 토큰 수를 계산하는 함수(모델: "gpt-3.5-turbo")
def calc_token_num(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    encoded_list = enc.encode(text) # 텍스트 인코딩해 인코딩 리스트 생성
    token_num= len(encoded_list)   # 인코딩 리스트의 길이로 토큰 개수 계산
    
    return token_num

[13장: 338페이지]

In [ ]:
ts_token_num = calc_token_num(yt_transcript)
print("- 자막의 토큰 개수:", ts_token_num)

In [ ]:
import textwrap

# 토큰에 따라 텍스트를 나눠 분할하는 함수
def divide_text(text, token_num):
    req_max_token = 2000 # 응답을 고려해 설정한 최대 요청 토큰
    
    divide_num = int(token_num/req_max_token) + 1 # 나눌 계수를 계산
    divide_char_num = int(len(text) / divide_num) # 나눌 문자 개수 
    divide_width =  divide_char_num + 20 # wrap() 함수로 텍스트 나눌 때 여유분 고려해 20 더함

    divided_text_list = textwrap.wrap(text, width=divide_width)
    
    return divide_num, divided_text_list

[13장: 339페이지]

In [ ]:
div_num, div_yt_transcripts = divide_text(yt_transcript, ts_token_num)

print("- 자막 그룹의 개수:", div_num)
print("- 나뉜 자막 리스트의 크기:", len(div_yt_transcripts))

for div_yt_transcript in div_yt_transcripts:
    div_token_num = calc_token_num(div_yt_transcript) # 텍스트의 토큰 수 계산
    print(" * 나뉜 자막의 토큰 개수:", div_token_num)

In [ ]:
import my_text_sum # 텍스트를 요약하기 위한 모듈

yt_summaries = []

for div_yt_transcript in div_yt_transcripts:
    yt_summary = my_text_sum.summarize_text(div_yt_transcript)
    yt_summaries.append(yt_summary)
    
print("- 요약 개수:", len(yt_summaries))
# print("- 페이지별 요약문:", yt_summaries)

[13장: 340페이지]

In [ ]:
import my_text_sum # 텍스트를 요약하기 위한 모듈

lang = 'en' # 영어로 요약
final_yt_token_num, final_yt_summary = my_text_sum.summarize_text_final(yt_summaries, lang)

if final_yt_summary != "":
    shorten_yt_final_summary = textwrap.shorten(final_yt_summary, 250,
                                                placeholder=' [..이하 생략..]')
    print("- 통합한 페이지 요약의 토큰 수:", final_yt_token_num)
    print("- 최종 요약(축약)\n", shorten_yt_final_summary) # 최종 요약문 출력(축약)
#     print("- 최종 요약\n", final_yt_summary) # 최종 요약문 출력

## 13.3 유튜브 동영상 요약 웹 앱

[13장: 341페이지]

In [ ]:
%%writefile C:\myPyAI\code\st\youtube_summary_app.py
# 유튜브 동영상을 요약하고 번역하는 웹 앱

import my_yt_tran  # 유튜브 동영상 정보와 자막을 가져오기 위한 모듈 임포트
import my_text_sum # 텍스트를 요약하기 위한 모듈
import streamlit as st
import openai
import os
import tiktoken
import textwrap
import deepl

# 텍스트의 토큰 수를 계산하는 함수(모델: "gpt-3.5-turbo")
def calc_token_num(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    encoded_list = enc.encode(text) # 텍스트 인코딩해 인코딩 리스트 생성
    token_num= len(encoded_list)    # 인코딩 리스트의 길이로 토큰 개수 계산
    
    return token_num

# 토큰에 따라 텍스트를 나눠 분할하는 함수
def divide_text(text, token_num):
    req_max_token = 2000 # 응답을 고려해 설정한 최대 요청 토큰
    
    divide_num = int(token_num/req_max_token) + 1 # 나눌 계수를 계산
    divide_char_num = int(len(text) / divide_num) # 나눌 문자 개수 
    divide_width =  divide_char_num + 20 # wrap() 함수로 텍스트 나눌 때 여유분 고려해 20 더함

    divided_text_list = textwrap.wrap(text, width=divide_width)
    
    return divide_num, divided_text_list

# 유튜브 동영상을 요약하는 함수
def summarize_youtube_video(video_url, selected_lang, trans_method):
    
    if selected_lang == '영어':
        lang = 'en' 
    else:
        lang = 'ko' 
    
    # 유튜브 동영상 플레이
    st.video(video_url, format='video/mp4') # st.video(video_url) 도 동일

    # 유튜브 동영상 제목 가져오기
    _, yt_title, _, _, yt_duration = my_yt_tran.get_youtube_video_info(video_url)
    st.write(f"[제목] {yt_title}, [길이(분:초)] {yt_duration}") # 제목 및 상영 시간출력
    
    # 유튜브 동영상 자막 가져오기
    yt_transcript = my_yt_tran.get_transcript_from_youtube(video_url, lang)

    # 자막 텍스트의 토큰 수 계산
    token_num = calc_token_num(yt_transcript)
    
    # 자막 텍스트를 분할해 리스트 생성
    div_num, divided_yt_transcripts = divide_text(yt_transcript, token_num)

    st.write("유튜브 동영상 내용 요약 중입니다. 잠시만 기다려 주세요.") 
    
    # 분할 자막의 요약 생성
    summaries = []
    for divided_yt_transcript in divided_yt_transcripts:
        summary = my_text_sum.summarize_text(divided_yt_transcript, lang) # 텍스트 요약
        summaries.append(summary)
        
    # 분할 자막의 요약을 다시 요약     
    _, final_summary = my_text_sum.summarize_text_final(summaries, lang)

    if selected_lang == '영어':
        shorten_num = 200 
    else:
        shorten_num = 120 
        
    shorten_final_summary = textwrap.shorten(final_summary, shorten_num, placeholder=' [..이하 생략..]')
    st.write("- 자막 요약(축약):", shorten_final_summary) # 최종 요약문 출력 (축약)
    # st.write("- 자막 요약:", final_summary) # 최종 요약문 출력

    if selected_lang == '영어': 
        if trans_method == 'OpenAI':
            trans_result = my_text_sum.traslate_english_to_korean_using_openAI(final_summary)
        elif trans_method == 'DeepL':
            trans_result = my_text_sum.traslate_english_to_korean_using_deepL(final_summary)

        shorten_trans_result = textwrap.shorten(trans_result, 120 ,placeholder=' [..이하 생략..]')
        st.write("- 한국어 요약(축약):", shorten_trans_result) # 한국어 번역문 출력 (축약)
        #st.write("- 한국어 요약:", trans_result) # 한국어 번역문 출력
        
# ------------------- 콜백 함수 --------------------
def button_callback():
    st.session_state['input'] = ""
    
# ------------- 사이드바 화면 구성 --------------------------  
st.sidebar.title("요약 설정 ")
url_text = st.sidebar.text_input("유튜브 동영상 URL을 입력하세요.", key="input")

clicked_for_clear = st.sidebar.button('URL 입력 내용 지우기',  on_click=button_callback)

yt_lang = st.sidebar.radio('유튜브 동영상 언어 선택', ['한국어', '영어'], index=1, horizontal=True)
    
if yt_lang == '영어':
    trans_method = st.sidebar.radio('번역 방법 선택', ['OpenAI', 'DeepL'], index=1, horizontal=True)
else:
    trans_method = ""

clicked_for_sum = st.sidebar.button('동영상 내용 요약')

# ------------- 메인 화면 구성 --------------------------     
st.title("유튜브 동영상 요약")

# 텍스트 입력이 있으면 수행
if url_text and clicked_for_sum: 
    yt_video_url = url_text.strip()
    summarize_youtube_video(yt_video_url, yt_lang, trans_method)

## 13.4 정리